# Using Citibike data and optimizing dataframes

## Dataset details
- Trip Duration (seconds)
- Start Time and Date
- Stop Time and Date
- Start Station Name
- End Station Name
- Station ID
- Station Lat/Long
- Bike ID
- User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
- Gender (Zero=unknown; 1=male; 2=female)
- Year of Birth

### Imports

In [44]:
import pandas as pd

<h2 style="background-color:green;color:white;padding:5px">If we know the datatypes, we can specify it directly at the time of loading</h3>

In [47]:
optimized_df = pd.read_csv('JC-201810-citibike-tripdata.csv',
                                 dtype={'tripduration': 'uint32',
                                        'start station id': 'uint16',
                                        'end station id': 'uint16',
                                        'bikeid': 'uint16',
                                        'birth year': 'uint16',
                                        'gender': 'uint8',
                                        'start station latitude': 'float32',
                                        'start station longitude': 'float32',
                                        'end station latitude': 'float32',
                                        'end station longitude': 'float32',
                                        'start station name': 'category',
                                        'end station name': 'category', 
                                        'usertype': 'category'
                                        },
                                 parse_dates=['starttime', 'stoptime'],
                                 infer_datetime_format=True)

print(optimized_df.shape)
optimized_df.head()

(39138, 15)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,152,2018-10-01 07:44:41.417,2018-10-01 07:47:14.296,3183,Exchange Place,40.716248,-74.033463,3639,Harborside,40.719254,-74.034233,15302,Subscriber,1971,2
1,122,2018-10-01 08:50:05.434,2018-10-01 08:52:08.225,3183,Exchange Place,40.716248,-74.033463,3214,Essex Light Rail,40.712772,-74.036484,29654,Subscriber,1973,1
2,211,2018-10-01 09:03:17.064,2018-10-01 09:06:48.094,3183,Exchange Place,40.716248,-74.033463,3187,Warren St,40.721123,-74.038048,33621,Subscriber,1958,1
3,342,2018-10-01 10:13:07.009,2018-10-01 10:18:49.896,3183,Exchange Place,40.716248,-74.033463,3202,Newport PATH,40.727222,-74.033760,26266,Subscriber,1982,1
4,2976,2018-10-01 10:45:14.280,2018-10-01 11:34:51.261,3183,Exchange Place,40.716248,-74.033463,3183,Exchange Place,40.716248,-74.033463,27083,Customer,1969,0


<h1 style="color: white; background-color:green;padding:5px"> But in many cases, we may not know the data ahead of time, or we could have different file types like pickle or HDF, where such options are not available during load</h1>
<h1>So we have to analyze the dataset, let us do that </h1>

In [48]:
df = pd.read_csv("JC-201810-citibike-tripdata.csv")

In [51]:
print(df.shape)
df.head()

(39138, 15)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,152,2018-10-01 07:44:41.4170,2018-10-01 07:47:14.2960,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,15302,Subscriber,1971,2
1,122,2018-10-01 08:50:05.4340,2018-10-01 08:52:08.2250,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,29654,Subscriber,1973,1
2,211,2018-10-01 09:03:17.0640,2018-10-01 09:06:48.0940,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.038051,33621,Subscriber,1958,1
3,342,2018-10-01 10:13:07.0090,2018-10-01 10:18:49.8960,3183,Exchange Place,40.716247,-74.033459,3202,Newport PATH,40.727224,-74.033759,26266,Subscriber,1982,1
4,2976,2018-10-01 10:45:14.2800,2018-10-01 11:34:51.2610,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,-74.033459,27083,Customer,1969,0


### Let us look at the various data types - Pandas has inferred them

In [4]:
df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

### Here is the total bytes used for each column

In [7]:
df.memory_usage(deep=True)

Index                           72
tripduration                313104
starttime                  2700522
stoptime                   2700522
start station id            313104
start station name         2255968
start station latitude      313104
start station longitude     313104
end station id              313104
end station name           2255834
end station latitude        313104
end station longitude       313104
bikeid                      313104
usertype                   2148582
birth year                  313104
gender                      313104
dtype: int64

### I wrote a function to get the total bytes of the dataframe, so it is easy to know

In [52]:
def get_memory_usage(pandas_object, unit='MB'):
    if unit == "KB":
        dividing_factor = 1024.0
    elif unit == "MB":
        dividing_factor = 1024.0 ** 2
    
    # Check if it is a Pandas dataframe
    if isinstance(pandas_object,pd.DataFrame):
        total_bytes = pandas_object.memory_usage(deep=True).sum()
    else: # This means it is a Pandas series
        total_bytes = pandas_object.memory_usage(deep=True)
        
    usage_units = total_bytes / dividing_factor
    
    return "%3.2f %s" % (usage_units, unit)

### Show the total memory used by the Citibike dataframe

In [54]:
print(get_memory_usage(df))

14.49 MB


### These are functions that will cast the various data types

In [80]:
def downcast_int_to_unsigned_int(df):
    converted_df = df.apply(pd.to_numeric,downcast='unsigned')        
    return converted_df

def downcast_float(df):
    converted_df = df.apply(pd.to_numeric,downcast='float')
    return converted_df

def cast_object_to_category(df):
    """
    Cast each column to a type specified in the column-type mapping
    """
    
    # If I don't make a copy, it complains with a warning
    # "... A value is trying to be set on a copy of a slice from a DataFrame"
    cat_df = df.copy()
    
    for column in list(cat_df.columns):
        cat_df[column] = cat_df[column].astype('category')
    return cat_df

def cast_object_to_datetime(df, format='%Y-%m-%d %H:%M:%S.%f'):
    """
    Cast each column to a type specified in the column-type mapping
    """    
    # If I don't make a copy, it complains with a warning
    # "... A value is trying to be set on a copy of a slice from a DataFrame"
    cat_df = df.copy()
    for column in list(cat_df.columns):
        cat_df[column] = pd.to_datetime(cat_df[column],format=format)
    return cat_df

def optimize_pandas_df(df, data_type):
    print("*** Before type conversion: ***")
    print("df%r \n\n* Memory usage = %s" % (list(df.columns), get_memory_usage(df)))
    print("* dtypes:\n%r" % df.dtypes)

    if data_type == "int":
        df = downcast_int_to_unsigned_int(df)
    elif data_type == "float":
        df = downcast_float(df)       
    elif data_type == "object":
        df = cast_object_to_category(df)
    elif data_type == "datetime":
        df = cast_object_to_datetime(df)
        
    print("\n\n*** After type conversion: ***")
    print("df%r \n\n* Memory usage = %s" % (list(df.columns), get_memory_usage(df)))
    print("* dtypes:\n%r" % df.dtypes)
    
    return df

### Make a copy of the dataframe, this copy will be continuously optimized for memory

In [81]:
copy_df = df.copy()

<h2 style="color: white; background-color:green;padding:5px">int columns: Look at all of them first</h2>

In [82]:
copy_df.select_dtypes(include=['int']).describe()

,tripduration,start station id,end station id,bikeid,birth year,gender
count,39138.000000,39138.000000,39138.000000,39138.000000,39138.000000,39138.000000
mean,589.920359,3273.602995,3265.638127,29251.220987,1981.062829,1.180106
std,3393.197160,147.852335,158.997274,2613.864275,10.130957,0.478995
min,61.000000,3183.000000,259.000000,14793.000000,1888.000000,0.000000
25%,225.000000,3191.000000,3186.000000,26315.000000,1975.000000,1.000000
50%,326.000000,3207.000000,3203.000000,29452.000000,1984.000000,1.000000
75%,522.750000,3273.000000,3272.000000,29640.000000,1989.000000,1.000000
max,328778.000000,3694.000000,3694.000000,35009.000000,2002.000000,2.000000


### Pick the columns to optimize

In [83]:
print(list(df.select_dtypes(include=['int']).columns))

['tripduration', 'start station id', 'end station id', 'bikeid', 'birth year', 'gender']


### Optimize them using downcasting

In [84]:
int_columns = ['tripduration', 'start station id', 'end station id', 'bikeid', 'birth year', 'gender']
copy_df[int_columns] = optimize_pandas_df(copy_df[int_columns], 'int')

*** Before type conversion: ***
df['tripduration', 'start station id', 'end station id', 'bikeid', 'birth year', 'gender'] 

* Memory usage = 1.79 MB
* dtypes:
tripduration        int64
start station id    int64
end station id      int64
bikeid              int64
birth year          int64
gender              int64
dtype: object


*** After type conversion: ***
df['tripduration', 'start station id', 'end station id', 'bikeid', 'birth year', 'gender'] 

* Memory usage = 0.49 MB
* dtypes:
tripduration        uint32
start station id    uint16
end station id      uint16
bikeid              uint16
birth year          uint16
gender               uint8
dtype: object


### Overall savings: Before and after

In [85]:
print("Before typecasting: df memory usage = %s" % get_memory_usage(df))
print("After typecasting: copy_df memory usage = %s" % get_memory_usage(copy_df))

Before typecasting: df memory usage = 14.49 MB
After typecasting: copy_df memory usage = 13.18 MB


<h2 style="color: white; background-color:green;padding:5px">float columns: Look at all of them first</h2>

In [86]:
copy_df.select_dtypes(include=['float']).describe()

,start station latitude,start station longitude,end station latitude,end station longitude
count,39138.000000,39138.000000,39138.000000,39138.000000
mean,40.722677,-74.046233,40.722260,-74.045611
std,0.007221,0.010860,0.007038,0.010770
min,40.709651,-74.083639,40.701221,-74.083639
25%,40.718355,-74.050656,40.717733,-74.050389
50%,40.721525,-74.043845,40.719586,-74.043117
75%,40.727224,-74.038051,40.726012,-74.037683
max,40.748716,-74.032108,40.775794,-73.976206


### Pick the columns to optimize

In [87]:
print(list(df.select_dtypes(include=['float']).columns))

['start station latitude', 'start station longitude', 'end station latitude', 'end station longitude']


### Optimize them using downcasting

In [88]:
float_columns = ['start station latitude', 'start station longitude', 'end station latitude', 'end station longitude']
copy_df[float_columns] = optimize_pandas_df(copy_df[float_columns], 'float')

*** Before type conversion: ***
df['start station latitude', 'start station longitude', 'end station latitude', 'end station longitude'] 

* Memory usage = 1.19 MB
* dtypes:
start station latitude     float64
start station longitude    float64
end station latitude       float64
end station longitude      float64
dtype: object


*** After type conversion: ***
df['start station latitude', 'start station longitude', 'end station latitude', 'end station longitude'] 

* Memory usage = 0.60 MB
* dtypes:
start station latitude     float32
start station longitude    float32
end station latitude       float32
end station longitude      float32
dtype: object


### Overall savings: Before and after

In [89]:
print("Before typecasting: df memory usage = %s" % get_memory_usage(df))
print("After typecasting: copy_df memory usage = %s" % get_memory_usage(copy_df))

Before typecasting: df memory usage = 14.49 MB
After typecasting: copy_df memory usage = 12.59 MB


<h2 style="color: white; background-color:green;padding:5px">object columns: Look at all of them first, focus on string columns</h2>

In [90]:
df.select_dtypes(include=['object']).head()

,starttime,stoptime,start station name,end station name,usertype
0,2018-10-01 07:44:41.4170,2018-10-01 07:47:14.2960,Exchange Place,Harborside,Subscriber
1,2018-10-01 08:50:05.4340,2018-10-01 08:52:08.2250,Exchange Place,Essex Light Rail,Subscriber
2,2018-10-01 09:03:17.0640,2018-10-01 09:06:48.0940,Exchange Place,Warren St,Subscriber
3,2018-10-01 10:13:07.0090,2018-10-01 10:18:49.8960,Exchange Place,Newport PATH,Subscriber
4,2018-10-01 10:45:14.2800,2018-10-01 11:34:51.2610,Exchange Place,Exchange Place,Customer


### There are only 2 unique usertypes, 51 unique start station names, and 57 unique end station names

In [91]:
df.select_dtypes(include=['object']).describe()

,starttime,stoptime,start station name,end station name,usertype
count,39138,39138,39138,39138,39138
unique,39137,39136,51,57,2
top,2018-10-15 17:07:56.1600,2018-10-31 18:18:17.4840,Grove St PATH,Grove St PATH,Subscriber
freq,2,2,4752,6156,37134


### Pick the columns to optimize, leave out the date time columns for now

In [92]:
print(list(df.select_dtypes(include=['object']).columns))

['starttime', 'stoptime', 'start station name', 'end station name', 'usertype']


### Cast them as type category

<h3 style="color: white; background-color:green;padding:5px">
    <ul>
        <li> This will internally build an integer-to-string mapping dictinary, so 1 integer per unique category</li>
        <li>Pandas uses the most space efficient integer subtype that can represent all of the unique values in a column</li>
    </ul></h3>

In [93]:
object_columns = ['start station name', 'end station name', 'usertype']

copy_df[object_columns] = optimize_pandas_df(copy_df[object_columns], 'object')

*** Before type conversion: ***
df['start station name', 'end station name', 'usertype'] 

* Memory usage = 6.35 MB
* dtypes:
start station name    object
end station name      object
usertype              object
dtype: object


*** After type conversion: ***
df['start station name', 'end station name', 'usertype'] 

* Memory usage = 0.12 MB
* dtypes:
start station name    category
end station name      category
usertype              category
dtype: object


### Overall savings: Before and after

In [95]:
print("Before type conversion: df memory usage = %s" % get_memory_usage(df))
print("After type conversion: copy_df memory usage = %s" % get_memory_usage(copy_df))

Before type conversion: df memory usage = 14.49 MB
After type conversion: copy_df memory usage = 6.36 MB


<h2 style="color: white; background-color:green;padding:5px">object columns: Look at all of them first, focus on datetime columns</h2>

In [96]:
datetime_columns = ['starttime', 'stoptime']

copy_df[datetime_columns] = optimize_pandas_df(copy_df[datetime_columns], 'datetime')

*** Before type conversion: ***
df['starttime', 'stoptime'] 

* Memory usage = 5.15 MB
* dtypes:
starttime    object
stoptime     object
dtype: object


*** After type conversion: ***
df['starttime', 'stoptime'] 

* Memory usage = 0.60 MB
* dtypes:
starttime    datetime64[ns]
stoptime     datetime64[ns]
dtype: object


### Overall savings: Before and after

In [97]:
print("Before type conversion: df memory usage = %s" % get_memory_usage(df))
print("After type conversion: copy_df memory usage = %s" % get_memory_usage(copy_df))

Before type conversion: df memory usage = 14.49 MB
After type conversion: copy_df memory usage = 1.80 MB


<hr style="height:2px" />

<h1 style="color: white; background-color:green;padding:5px">Libraries are being developed to scale Pandas dataframes beyond 1 machine, Dask is one such library</h1>

## Does Dask optimize the dataframe? Let us check

In [100]:
import dask.dataframe as dd

### Read the data

In [102]:
df_dd = dd.read_csv("JC-201810-citibike-tripdata.csv")

### Check the memory usage..hmm..seems good, but not as good as what we had above..1.8 MB

In [110]:
df_dd.info(memory_usage='deep')

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, tripduration to gender
dtypes: object(5), float64(4), int64(6)
memory usage: 4.5 MB


### Let us feed our Citbike dataframe to Dask

In [111]:
copy_df_dd = dd.from_pandas(copy_df, npartitions=1)
print(copy_df.shape)
copy_df_dd.head()

(39138, 15)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,152,2018-10-01 07:44:41.417,2018-10-01 07:47:14.296,3183,Exchange Place,40.716248,-74.033463,3639,Harborside,40.719254,-74.034233,15302,Subscriber,1971,2
1,122,2018-10-01 08:50:05.434,2018-10-01 08:52:08.225,3183,Exchange Place,40.716248,-74.033463,3214,Essex Light Rail,40.712772,-74.036484,29654,Subscriber,1973,1
2,211,2018-10-01 09:03:17.064,2018-10-01 09:06:48.094,3183,Exchange Place,40.716248,-74.033463,3187,Warren St,40.721123,-74.038048,33621,Subscriber,1958,1
3,342,2018-10-01 10:13:07.009,2018-10-01 10:18:49.896,3183,Exchange Place,40.716248,-74.033463,3202,Newport PATH,40.727222,-74.033760,26266,Subscriber,1982,1
4,2976,2018-10-01 10:45:14.280,2018-10-01 11:34:51.261,3183,Exchange Place,40.716248,-74.033463,3183,Exchange Place,40.716248,-74.033463,27083,Customer,1969,0


### Check the memory usage..well, Dask has accepted our optimized Dataframe..WOOHOO! :)

In [112]:
copy_df_dd.info(memory_usage='deep')

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, tripduration to gender
dtypes: category(3), datetime64[ns](2), float32(4), uint16(4), uint32(1), uint8(1)
memory usage: 1.8 MB
